In [1]:
! pip install vegas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 29.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 57.8 MB/s eta 0:00:00a 0:00:01



Tutorial
=======================================

Run through the tutorial at https://vegas.readthedocs.io/en/latest/tutorial.html. Some of the code is copied here already.

.. moduleauthor:: G. Peter Lepage <g.p.lepage@cornell.edu>

.. |Integrator| replace:: :class:`vegas.Integrator`
.. |PDFIntegrator| replace:: :class:`vegas.PDFIntegrator`
.. |AdaptiveMap| replace:: :class:`vegas.AdaptiveMap`
.. |vegas| replace:: :mod:`vegas`
.. |WAvg| replace:: :class:`vegas.RunningWAvg`
.. |chi2| replace:: :math:`\chi^2`
.. |x| replace:: x
.. |y| replace:: y
.. |~| unicode:: U+00A0
.. |times| unicode:: U+00D7
   :trim:

Introduction
-------------

Class :class:`vegas.Integrator` gives Monte Carlo estimates of arbitrary
multidimensional integrals using the *vegas* algorithm
(G. P. Lepage, J. Comput. Phys. 27 (1978) 192 and J. Comput. Phys. 439 (2021) 110386).
The algorithm has two components.
First an automatic transformation is applied to to the integration variables
in an attempt to flatten the integrand. Then a Monte Carlo estimate of the
integral is made using the  transformed variables. Flattening the integrand
makes the integral easier and improves the estimate.  The transformation
applied to the integration variables is optimized
over several iterations of the algorithm: information about the integrand that
is collected during one iteration is used to  improve the transformation used
in the next iteration.

Monte Carlo integration makes few assumptions about the
integrand --- it needn't be analytic nor even continuous. This
makes Monte Carlo integration unusually robust. It also makes it well suited
for adaptive integration. Adaptive strategies are essential for
multidimensional integration, especially in high dimensions, because
multidimensional space is large, with  lots of corners, making it
easy to lose important features in the integrand.

Monte Carlo integration also provides efficient and reliable methods for
estimating the
accuracy of its results. In particular, each Monte Carlo
estimate of an integral is a random number from a distribution
whose mean is the correct value of the integral. This distribution is
Gaussian or normal provided
the number of integrand samples is sufficiently large.
In practice we generate multiple
estimates of the integral
in order to verify that the distribution is indeed Gaussian.
Error analysis is straightforward if the
integral estimates are Gaussian.

The |vegas| algorithm has been in use for decades and implementations are
available in many programming languages, including Fortran (the original
version), C and C++. The algorithm used here is significantly improved over
the original implementation, and that used in most other implementations.
It uses two adaptive strategies: importance sampling, as in the original
implementation, and adaptive stratified sampling, which is new. The 
new algorithm is described in G. P. Lepage, arXiv_2009.05112_ 
(J. Comput. Phys. 439 (2021) 110386).

.. _arXiv_2009.05112: https://arxiv.org/abs/2009.05112

There is also a third adaptive strategy, adaptive re-stratification, that can 
be useful in very high dimensions. See :func:`vegas.restratify`.

This module is written in Cython, so it is almost as fast as compiled Fortran or
C, particularly when the integrand is also coded in Cython (or some other
compiled language), as discussed below.

The following sections describe how to use |vegas|. Almost every
example shown is a complete code, which can be copied into a file
and run with Python. It is worthwhile playing with the parameters to see how
things change.


Basic Integrals
----------------
Here we illustrate the use of |vegas| by estimating the integral

$$
    C\int_{-1}^1 dx_0 \int_0^1 dx_1 \int_0^1 dx_2 \int_0^1 dx_3
    \,\,\mathrm{e}^{- 100 \sum_{d}(x_d-0.5)^2}  ,
$$
where constant $C$ is chosen so that the exact integral is 1.
The following code shows how this can be done::


In [3]:
import vegas
import math

In [4]:
def f(x):
    dx2 = 0
    for d in range(4):
        dx2 += (x[d] - 0.5) ** 2
    return math.exp(-dx2 * 100.) * 1013.2118364296088

integ = vegas.Integrator([[-1, 1], [0, 1], [0, 1], [0, 1]])

result = integ(f, nitn=10, neval=1000)
print(result.summary())
print('result = %s    Q = %.2f' % (result, result.Q))


itn   integral        wgt average     chi2/dof        Q
-------------------------------------------------------
  1   0.92(57)        0.92(57)            0.00     1.00
  2   1.12(19)        1.10(18)            0.11     0.75
  3   0.99(12)        1.022(99)           0.19     0.83
  4   1.067(55)       1.056(48)           0.18     0.91
  5   1.002(39)       1.024(30)           0.32     0.86
  6   1.006(25)       1.014(19)           0.30     0.91
  7   0.997(19)       1.005(13)           0.31     0.93
  8   1.009(15)       1.007(10)           0.27     0.97
  9   0.994(12)       1.0018(78)          0.32     0.96
 10   0.998(11)       1.0006(64)          0.29     0.98

result = 1.0006(64)    Q = 0.98
